In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials 

In [ ]:
SCOPES = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'あなたのアカウントサービス.json'

In [ ]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, SCOPES)

In [ ]:
gs = gspread.authorize(credentials)

SPREADSHEET_KEY = 'あなたのスプレッドシートキー'
worksheet = gs.open_by_key(SPREADSHEET_KEY).worksheet("シート名")

In [ ]:
print(worksheet.acell("G2").value)

In [ ]:
import pandas as pd

In [ ]:
workbook = gs.open_by_key(SPREADSHEET_KEY)
worksheet = workbook.worksheet("シート名")

In [ ]:
print(workbook.title)
print(workbook.id)

In [ ]:
print(worksheet)

In [ ]:
df = pd.DataFrame(worksheet.get_all_values())
df.head()

In [ ]:
df.columns = df.iloc[0]
df.head()

In [ ]:
df = df.drop(df.index[[0]])
df.head()

In [ ]:
df.dtypes

In [ ]:
df['発注金額'] = df['発注金額'].str.replace(",", "")
df.head()

In [ ]:
df['発注金額'] = df['発注金額'].astype(int)

In [ ]:
df.dtypes

In [ ]:
df_sum = df[['会社名','発注金額']].groupby("会社名").sum()
df_sum.head()

In [ ]:
from gspread_dataframe import set_with_dataframe

In [ ]:
workbook.add_worksheet(title="会社別発注", rows=50, cols=10)

In [ ]:
set_with_dataframe(workbook.worksheet("会社別発注"), df_sum, include_index=True)

In [ ]:
workbook.add_worksheet(title="会社別発注", rows=50, cols=10)

In [ ]:
workbook.del_worksheet(workbook.worksheet("会社別発注"))

In [ ]:
workbook = gs.open_by_key(SPREADSHEET_KEY)
worksheet = workbook.worksheet("シート名")

In [ ]:
worksheet.get_all_values()[0:5]

In [ ]:
df = pd.DataFrame(worksheet.get_all_values()[1:],columns=worksheet.get_all_values()[0])
df.head()

In [ ]:
df['売上金額'] = df['売上金額'].str.replace(",", "").astype(int)
df.head()

In [ ]:
df_pivot01 = df.pivot_table(index='支店',values='売上金額', aggfunc='sum').applymap('{:,.0f}'.format)
df_pivot01

In [ ]:
df_pivot02 = df.pivot_table(index='商品名',values='売上金額', aggfunc='sum').applymap('{:,.0f}'.format)
df_pivot02

In [ ]:
df_pivot03 = df.pivot_table(index='氏名',values='売上金額', aggfunc='sum').applymap('{:,.0f}'.format)
df_pivot03

In [ ]:
df_pivot04 = df.pivot_table(index='氏名', columns='商品分類' ,values='売上金額', aggfunc='sum',fill_value=0).applymap('{:,.0f}'.format)
df_pivot04

In [ ]:
df_pivot05 = df.pivot_table(index=['氏名','売上日'], columns='商品分類' ,values='売上金額', aggfunc='sum',fill_value=0).applymap('{:,.0f}'.format)
df_pivot05

In [ ]:
workbook.add_worksheet(title="支店別売上", rows=200, cols=10)
set_with_dataframe(workbook.worksheet("支店別売上"), df_pivot01, include_index=True)

workbook.add_worksheet(title="商品名別売上", rows=200, cols=10)
set_with_dataframe(workbook.worksheet("商品名別売上"), df_pivot02, include_index=True)

workbook.add_worksheet(title="担当者別売上", rows=200, cols=10)
set_with_dataframe(workbook.worksheet("担当者別売上"), df_pivot03, include_index=True)

workbook.add_worksheet(title="担当者×商品分類×売上", rows=200, cols=10)
set_with_dataframe(workbook.worksheet("担当者×商品分類×売上"), df_pivot04, include_index=True)

workbook.add_worksheet(title="担当者×商品分類×売上日×売上", rows=200, cols=10)
set_with_dataframe(workbook.worksheet("担当者×商品分類×売上日×売上"), df_pivot05, include_index=True)